In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_csv('train.tsv', sep='\t') #분석에 사용할 데이터.
test_df1 = pd.read_csv('test.tsv', sep='\t') #종속변수(y)값 예측하는 모델을 만든 후, 적용 할 데이터.

In [ ]:
print(train_df.shape)

train_df.tail()

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
train_df.isna().sum()

In [ ]:
print(mercari_df.isna().sum())
# 중복되는 상품 이름 찾기
duplicated_name_train=mercari_df[mercari_df['name'].duplicated()]['name'].unique()

# 중복된 이름이면서 'category_name'이 nan값이 아닌 행들에서, 'name','category_name'만 추출, 하나의 이름만 남겨놓고 중복 제거, 이름을 인덱스로 설정, 'category_name'만 추출해서 Series로 반환
duplicated_category=mercari_df[(mercari_df['name'].isin(duplicated_name_train))&(mercari_df['category_name'].notna())][['name','category_name']].drop_duplicates(subset='name').set_index('name')['category_name']
# 'category_name'에서 결측값이 있는 행을 채운다, 어떤걸로? 반환된 시리즈를 결측치가 있는 mercari_df['name']을 키 값으로 찾아서
mercari_df['category_name']=mercari_df['category_name'].fillna(mercari_df['name'].map(duplicated_category))

# 위와 같은 작업, brand_name
duplicated_brand=mercari_df[(mercari_df['name'].isin(duplicated_name_train))&(mercari_df['brand_name'].notna())][['name','brand_name']].drop_duplicates(subset='name').set_index('name')['brand_name']
mercari_df['brand_name']=mercari_df['brand_name'].fillna(mercari_df['name'].map(duplicated_brand))

print(mercari_df.isna().sum())
# 브랜드 : 1203개 결측치 제거
# 카테고리 : 63773개 결측치 제거
# 이렇게 해도 안 되는 건 결측치 처리

In [ ]:
## 1. 금액의 분포에 대해서 파악했음.

plt.figure(figsize=(8, 4))
sns.histplot(train_df['price'], bins=50)
plt.xlim(0, 300)
plt.show()


#대부분의 제품이 5000엔 이하로 몰려있으며 고가의 상품은 매우 극소수로 존재함.
#긴 꼬리형태 모델은 회귀모델에 적합하지 않음. 히스토그램 log로 변환.

In [ ]:
train_df['log_price'] = np.log1p(train_df['price']) #log_price = in(price+1)

plt.figure(figsize=(8, 4))
sns.histplot(train_df['log_price'], bins=50)
plt.show()

#log_price = 2.5와 log_price = 3.5 사이에 위치하므로 11~32달러 사이의 금액이 많은 비중을 차지함을 알 수 있다.

In [ ]:
train_df['category_name'] = train_df['category_name'].fillna('Unknown')

train_df['category_split'] = train_df['category_name'].str.split('/')

train_df['category_first'] = train_df['category_split'].str[0]

train_df['category_second'] = train_df['category_split'].str[1]

train_df['category_third'] = train_df['category_split'].str[2]

train_df.head()

In [ ]:
train_df['category_first'] = train_df['category_first'].astype(str).str.lower()
train_df['category_second'] = train_df['category_second'].astype(str).str.lower()
train_df['category_third'] = train_df['category_third'].astype(str).str.lower()

train_df['category_first'] = train_df['category_first'].str.replace("'s", "", regex=False)
train_df['category_second'] = train_df['category_second'].str.replace("'s", "", regex=False)
train_df['category_third'] = train_df['category_third'].str.replace("'s", "", regex=False)

In [ ]:
print(train_df['category_first'].nunique())

print(train_df['category_second'].nunique())

print(train_df['category_third'].nunique())

In [ ]:
#카테고리별에 따른 금액 비교
plt.figure(figsize=(12,4))
sns.barplot(x=train_df['category_first'], y=train_df['price'])
plt.xticks(rotation=45)
plt.show()

#plt.figure(figsize=(12,4))
#sns.barplot(x=train_df['category_second'], y=train_df['price'])
#plt.xticks(rotation=45)
#plt.show()

#plt.figure(figsize=(12,4))
#sns.barplot(x=train_df['category_third'], y=train_df['price'])
#plt.xticks(rotation=45)
#plt.show()

In [ ]:
#